In [1]:
'''
function to train GaussianNB classifier
'''
def classify(features_train, labels_train):
    from sklearn.naive_bayes import GaussianNB
    clf = GaussianNB()
    clf.fit(features_train, labels_train)
    return clf 

generate data

In [2]:
import random

def makeTerrainData(n_points=1000):
      random.seed(42)
      grade = [random.random() for ii in range(0,n_points)]
      bumpy = [random.random() for ii in range(0,n_points)]
      error = [random.random() for ii in range(0,n_points)]
      
      y = [round(grade[ii]*bumpy[ii]+0.3+0.1*error[ii]) for ii in range(0,n_points)]
      
      for ii in range(0, len(y)):
         if grade[ii]>0.8 or bumpy[ii]>0.8:
               y[ii] = 1.0
      
      # split into train/test sets
      x = [[gg, ss] for gg, ss in zip(grade, bumpy)]
      split = int(0.75*n_points)
      x_train = x[0:split]
      x_test = x[split:]
      y_train = y[0:split]
      y_test = y[split:]

      grade_sig = [x_train[ii][0] for ii in range(0, len(x_train)) if y_train[ii]==0]
      bumpy_sig = [x_train[ii][1] for ii in range(0, len(x_train)) if y_train[ii]==0]
      grade_bkg = [x_train[ii][0] for ii in range(0, len(x_train)) if y_train[ii]==1]
      bumpy_bkg = [x_train[ii][1] for ii in range(0, len(x_train)) if y_train[ii]==1]

      training_data = {"fast":{"grade":grade_sig, "bumpiness":bumpy_sig}
            , "slow":{"grade":grade_bkg, "bumpiness":bumpy_bkg}}
      
      grade_sig = [x_test[ii][0] for ii in range(0, len(x_test)) if y_test[ii]==0]
      bumpy_sig = [x_test[ii][1] for ii in range(0, len(x_test)) if y_test[ii]==0]
      grade_bkg = [x_test[ii][0] for ii in range(0, len(x_test)) if y_test[ii]==1]
      bumpy_bkg = [x_test[ii][1] for ii in range(0, len(x_test)) if y_test[ii]==1]

      test_data = {"fast":{"grade":grade_sig, "bumpiness":bumpy_sig}
            , "slow":{"grade":grade_bkg, "bumpiness":bumpy_bkg}}
      

      return x_train, y_train, x_test, y_test
      #return training_data, test_data

classification visualizer

In [7]:
# from udacityplots import *
import warnings
warnings.filterwarnings("ignore")

import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
import pylab as pl
import numpy as np

# import numpy as np
# import matplotlib.pyplot as plt
# plt.ioff()

def prettyPicture(clf, x_test, y_test):
   x_min = 0.0; x_max = 1.0
   y_min = 0.0; y_max = 1.0

   # Plot the decision boundary. for that, we will assign a color to each
   # pount in the mesh [x_min, x_max]x[y_min, y_max]
   h = .01
   xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
   z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

   # Put the result into a color plot
   z = z.reshape(xx.shape)
   plt.xlim(xx.min(), xx.max())
   plt.ylim(yy.min(), yy.max())

   plt.pcolormesh(xx, yy, z, cmap=pl.cm.seismic)

   # plot also the test points
   grade_sig = [x_test[ii][0] for ii in range(0, len(x_test)) if y_test[ii]==0]
   bumpy_sig = [x_test[ii][1] for ii in range(0, len(x_test)) if y_test[ii]==0]
   grade_bkg = [x_test[ii][0] for ii in range(0, len(x_test)) if y_test[ii]==1]
   bumpy_bkg = [x_test[ii][1] for ii in range(0, len(x_test)) if y_test[ii]==1]

   plt.scatter(grade_sig, bumpy_sig, color = "b", label = "fast")
   plt.scatter(grade_bkg, bumpy_bkg, color="r", label = "slow")
   plt.legend()
   plt.xlabel("bumpiness")
   plt.ylabel("grade")
   plt.savefig("test.png")

import base64
import json
import subprocess

def outputImage(name, format, bytes):
   image_start = "BEGIN_IMAGE_f9825uweof8jw9fj4r8"
   image_end = "END_IMAGE_0238jfw08fjsiufhw8frs"
   data = {}
   data['name'] = name
   data['format'] = format
   data['bytes'] = base64.encode(bytes)
   print (image_start+json.dumps(data)+image_end)


run the codes

In [8]:
features_train, labels_train, features_test, labels_test = makeTerrainData()

### the training data (features_train, labels_train) have both "fast" and "slow" points mixed
### in together--separate them so we can give them different colors in the scatterplot,
### and visually identify them
grade_fast = [features_train[ii][0] for ii in range(0, len(features_train)) if labels_train[ii]==0]
bumpy_fast = [features_train[ii][1] for ii in range(0, len(features_train)) if labels_train[ii]==0]
grade_slow = [features_train[ii][0] for ii in range(0, len(features_train)) if labels_train[ii]==1]
bumpy_slow = [features_train[ii][1] for ii in range(0, len(features_train)) if labels_train[ii]==1]

clf = classify(features_train, labels_train)

draw the decision boundary


In [10]:
prettyPicture(clf, features_test, labels_test)
# outputImage("test.png", "png", open("test.png", "rb").read())